<a href="https://colab.research.google.com/github/PedroAvilaPaiDaManu/Pedrocanizel/blob/main/project_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark pandas numpy

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.2.2-bin-hadoop3.2')


from pyspark.sql import functions as func
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import warnings, re


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


# Instanciando Spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

--2023-03-07 16:46:59--  https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz.1’

spark-3.2.2-bin-had 100%[===================>] 287.16M  9.58MB/s    in 20s     

2023-03-07 16:47:19 (14.2 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz.1’ saved [301112604/301112604]



In [177]:
import datetime

In [122]:
import seaborn as sns

In [70]:
from pyspark.sql.functions import monotonically_increasing_id 

In [71]:
path = "/content/Base de Dados - Questionário.xlsx - Sheet1.csv"

In [72]:
df = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=',').load(path)

In [73]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [74]:
df_index = df.select("*").withColumn("id", monotonically_increasing_id())

In [75]:
filtro = df_index['id'] > 2

In [76]:
df_novo = df_index.filter(filtro)

In [77]:
df_novo.select('_c0','_c1').distinct().show()

+----+----+
| _c0| _c1|
+----+----+
|null|null|
+----+----+



In [78]:
df_novo = df_novo.drop('_c0','_c1')
    

In [79]:
df_novo1 = df_novo.withColumnRenamed("_c2","idem")\
        .withColumnRenamed("_c3","product_id")\
        .withColumnRenamed("_c4","client_id")\
        .withColumnRenamed("_c5","discount")\
        .withColumnRenamed("_c6","unit_price")\
        .withColumnRenamed("_c7","quantity")\
        .withColumnRenamed("_c8","store_id")\
        .withColumnRenamed("_c9","date")\
        .withColumnRenamed("id","id_temp")

In [80]:
df_novo1 = df_novo1.filter(df_novo1.id_temp != 3)



In [118]:
df_novo1.groupBy("product_id").count().orderBy(F.col('count').desc()).show(truncate=False)

+---------------------------------+-----+
|product_id                       |count|
+---------------------------------+-----+
|002ec297b1b00fb9dde7ee6ac24b67713|966  |
|002d4ea7c04739c130bb74d7e7cd16943|963  |
|001c5d71ac6ad696d22315953758fa043|944  |
|0011c512eb256aa0dbbb544d8dffcf6e3|940  |
|0030026a6ddb3b2d1d4bc225b4b4c4da3|939  |
|002c6dab60557c48cfd6c2222ef7fd763|938  |
|001b237c0e9bb435f2e54071129237e93|935  |
|00126f27c813603687e6ce486d909d013|933  |
|0009406fd7479715e4bef61dd91f24623|927  |
|0021a87d4997a48b6cef1665602be0f53|922  |
|002959d7a0b0990fe2d69988affcbc803|920  |
|00088930e925c41fd95ebfe695fd26553|916  |
|001b72dfd63e9833e8c02742adf472e33|913  |
|002af88741ba70c7b5cf4e4a0ad7ef853|908  |
|0030e635639c898b323826589761cf233|907  |
|002552c0663708129c0019cc97552d7d3|903  |
|001795ec6f1b187d37335e1c4704762e3|900  |
|00250175f79f584c14ab5cecd80553cd3|895  |
|002159fe700ed3521f46cfcf6e941c763|893  |
|000d9be29b5207b54e86aa1b1ac548723|891  |
+---------------------------------

In [81]:
df_novo = df_novo.filter(filtro)

In [98]:
df_contagem = df_novo1.select("product_id").distinct().count()

In [99]:
df_contagem

73

In [105]:
df_contagem2 = df_novo1.groupBy("product_id")

In [124]:
df_5 = (
    df_novo1.withColumn('ano', F.substring('date', 1, 4).cast('integer'))
    )

In [194]:
df_ir= df_novo1.filter(df_novo1.date.contains('2019'))

In [208]:
pa_pf = df_ir.toPandas()


In [209]:
pa_pf["quantity"] = pd.to_numeric(pa_pf["quantity"])


In [210]:
pa_pf["date"] = pd.to_datetime(pa_pf["date"])


In [211]:
pa_pf["unit_price"] = pa_pf["unit_price"].str.replace(',','.').astype(float)


In [212]:
pa_pf["discount"] = pa_pf["discount"].str.replace(',','.').astype(float)

In [213]:
pa_pf['mes'] = pa_pf['date'].dt.month

In [214]:
pa_pf.head()

,idem,product_id,client_id,discount,unit_price,quantity,store_id,date,id_temp,mes
0,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,0.10,162.4,1,4,2019-04-25,5,4
1,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,0.10,194.6,1,4,2019-07-17,6,7
2,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,0.10,406.0,1,1,2019-01-21,8,1
3,7ab64f42a8f9ab290e920eed320d7e60,002552c0663708129c0019cc97552d7d3,14002,0.10,249.2,1,1,2019-03-28,9,3
4,b35a771164a9defa3b4767f1b66918b1,002959d7a0b0990fe2d69988affcbc803,14002,0.08,261.8,1,1,2019-08-14,10,8


In [215]:
pa_pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14768 entries, 0 to 14767
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   idem        14768 non-null  object        
 1   product_id  14768 non-null  object        
 2   client_id   14768 non-null  object        
 3   discount    14768 non-null  float64       
 4   unit_price  14768 non-null  float64       
 5   quantity    14768 non-null  int64         
 6   store_id    14768 non-null  object        
 7   date        14768 non-null  datetime64[ns]
 8   id_temp     14768 non-null  int64         
 9   mes         14768 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 1.1+ MB


In [216]:
pa_pf['liquida'] = (pa_pf['quantity'] * pa_pf['unit_price']) - pa_pf['discount'] * (pa_pf['quantity'] * pa_pf['unit_price'])

In [217]:
pa_pf.head()

,idem,product_id,client_id,discount,unit_price,quantity,store_id,date,id_temp,mes,liquida
0,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,0.10,162.4,1,4,2019-04-25,5,4,146.160
1,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,0.10,194.6,1,4,2019-07-17,6,7,175.140
2,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,0.10,406.0,1,1,2019-01-21,8,1,365.400
3,7ab64f42a8f9ab290e920eed320d7e60,002552c0663708129c0019cc97552d7d3,14002,0.10,249.2,1,1,2019-03-28,9,3,224.280
4,b35a771164a9defa3b4767f1b66918b1,002959d7a0b0990fe2d69988affcbc803,14002,0.08,261.8,1,1,2019-08-14,10,8,240.856


In [218]:
df_final = pa_pf[['mes','unit_price', 'liquida', 'quantity']].groupby('mes').sum().reset_index()

In [219]:
df_final.head()

,mes,unit_price,liquida,quantity
0,1,224638.4,210849.66,1057
1,2,178669.4,168483.42,834
2,3,303454.2,285083.82,1186
3,4,277320.4,257972.40,1078
4,5,300983.2,283353.84,1165


In [220]:
df_final.describe()

,mes,unit_price,liquida,quantity
count,12.000000,12.000000,12.000000,12.000000
mean,6.500000,309649.200000,294265.094667,1284.333333
std,3.605551,99616.642504,99304.561130,431.467124
min,1.000000,178669.400000,168483.420000,834.000000
25%,3.750000,256883.200000,238564.935000,1049.750000
50%,6.500000,273622.300000,257390.910000,1073.500000
75%,9.250000,341443.550000,323938.797000,1371.250000
max,12.000000,471296.000000,456192.856000,2035.000000


In [221]:
df_final_09 = pa_pf[['store_id', 'unit_price', 'liquida', 'quantity']].groupby('store_id').sum().reset_index()

In [222]:
df_final_09.describe()

,unit_price,liquida,quantity
count,7.00000,7.000000,7.000000
mean,530827.20000,504454.448000,2201.714286
std,305375.68204,290947.120527,1272.009921
min,107962.40000,102130.784000,444.000000
25%,303950.50000,288585.164000,1262.000000
50%,590185.40000,560306.096000,2401.000000
75%,732412.10000,695728.684000,3057.500000
max,944917.40000,900116.560000,3928.000000


In [225]:
display(df_final_09)

,store_id,unit_price,liquida,quantity
0,1,944917.4,900116.560,3928
1,2,760799.2,723401.616,3180
2,22,107962.4,102130.784,444
3,3,704025.0,668055.752,2935
4,4,590185.4,560306.096,2401
5,7,403666.2,384675.676,1673
6,9,204234.8,192494.652,851


In [167]:
type(pa_pf['discount'])

pandas.core.series.Series

In [190]:
df_final['liquida'] = (df_final['quantity'] * df_final['unit_price']) - df_final['discount'] * (df_final['quantity'] * df_final['discount'])

In [193]:
df_final.head()

,mes,unit_price,discount,quantity,liquida
0,1,224638.4,101.2,1057,2.266176e+08
1,2,178669.4,80.0,834,1.436727e+08
2,3,303454.2,115.1,1186,3.441845e+08
3,4,277320.4,104.2,1078,2.872469e+08
4,5,300983.2,111.9,1165,3.360578e+08


In [ ]:
df.mean(pa_pf) #Por padrão o mean no pandas faz a média das linhas

In [138]:
pa_pf.sort_values(by = 'date', inplace = True)

In [191]:
df_final.describe()

,mes,unit_price,discount,quantity,liquida
count,12.000000,12.000000,12.000000,12.000000,1.200000e+01
mean,6.500000,309649.200000,113.621667,1284.333333,4.168981e+08
std,3.605551,99616.642504,24.317726,431.467124,2.829750e+08
min,1.000000,178669.400000,80.000000,834.000000,1.436727e+08
25%,3.750000,256883.200000,100.975000,1049.750000,2.523908e+08
50%,6.500000,273622.300000,103.950000,1073.500000,2.821750e+08
75%,9.250000,341443.550000,123.025000,1371.250000,4.671511e+08
max,12.000000,471296.000000,153.760000,2035.000000,9.050497e+08


In [129]:
df_ir.show()

+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+
|                idem|          product_id|client_id|discount|unit_price|quantity|store_id|      date|id_temp|
+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+
|a97c9b194a026ac3e...|002d4ea7c04739c13...|    14074|    0,08|       210|       1|       2| 6/28/2019|    207|
|ea110ced6459b4dc6...|002d4ea7c04739c13...|    14080|     0,1|       210|       1|       1|  9/3/2019|    225|
|89fb8b371b28dcc03...|002d4ea7c04739c13...|    14088|     0,1|       210|       1|       1|  5/6/2019|    242|
|51efa345a1180ca38...|002d4ea7c04739c13...|    14127|     0,1|       210|       1|      22|  9/4/2019|    327|
|f89525a2a79be7efb...|002d4ea7c04739c13...|    14197|     0,1|       210|       1|       1|10/26/2019|    491|
|6ee1c0ae2850fce09...|002d4ea7c04739c13...|    14215|     0,1|       210|       1|       1|  1/2/2019|    533|
|

In [125]:
df_5.show()

+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+----+
|                idem|          product_id|client_id|discount|unit_price|quantity|store_id|      date|id_temp| ano|
+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+----+
|80260d682079b6090...|002552c0663708129...|    14001|    0,08|     249,2|       1|       4|12/10/2018|      4|null|
|3275736da1234f55d...|001b237c0e9bb435f...|    14001|     0,1|     162,4|       1|       4| 4/25/2019|      5|null|
|f3513eec8f1434b5b...|00066f42aeeb9f300...|    14001|     0,1|     194,6|       1|       4| 7/17/2019|      6|null|
|7180475b60fe6bbbd...|001b72dfd63e9833e...|    14002|     0,1|     201,6|       2|       1| 4/10/2018|      7|null|
|76600853a200fe87e...|00210e41887c2a8ef...|    14002|     0,1|       406|       1|       1| 1/21/2019|      8|null|
|7ab64f42a8f9ab290...|002552c0663708129...|    14002|     0,1|     249,2

In [82]:
df_novo1.show(20)

+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+
|                idem|          product_id|client_id|discount|unit_price|quantity|store_id|      date|id_temp|
+--------------------+--------------------+---------+--------+----------+--------+--------+----------+-------+
|80260d682079b6090...|002552c0663708129...|    14001|    0,08|     249,2|       1|       4|12/10/2018|      4|
|3275736da1234f55d...|001b237c0e9bb435f...|    14001|     0,1|     162,4|       1|       4| 4/25/2019|      5|
|f3513eec8f1434b5b...|00066f42aeeb9f300...|    14001|     0,1|     194,6|       1|       4| 7/17/2019|      6|
|7180475b60fe6bbbd...|001b72dfd63e9833e...|    14002|     0,1|     201,6|       2|       1| 4/10/2018|      7|
|76600853a200fe87e...|00210e41887c2a8ef...|    14002|     0,1|       406|       1|       1| 1/21/2019|      8|
|7ab64f42a8f9ab290...|002552c0663708129...|    14002|     0,1|     249,2|       1|       1| 3/28/2019|      9|
|

In [83]:
path2 = "/content/sheet_2.xlsx - Sheet2.csv"

In [84]:
df2 = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=',').load(path2)

In [92]:
df2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Size: string (nullable = true)



In [119]:
id_marca = '002ec297b1b00fb9dde7ee6ac24b67713'

In [127]:

df2.where(df2.Name == 'Iron Man').show(truncate=False)

+---------------------------------+--------+----+
|ID                               |Name    |Size|
+---------------------------------+--------+----+
|002d4ea7c04739c130bb74d7e7cd16941|Iron Man|P   |
|002d4ea7c04739c130bb74d7e7cd16942|Iron Man|M   |
|002d4ea7c04739c130bb74d7e7cd16943|Iron Man|G   |
+---------------------------------+--------+----+



In [95]:
df2.show(20, truncate=False)


+---------------------------------+---------------+----+
|ID                               |Name           |Size|
+---------------------------------+---------------+----+
|00066f42aeeb9f3007548bb9d3f33c381|Capitão América|P   |
|00066f42aeeb9f3007548bb9d3f33c382|Capitão América|M   |
|00066f42aeeb9f3007548bb9d3f33c383|Capitão América|G   |
|00088930e925c41fd95ebfe695fd26551|Thor           |P   |
|00088930e925c41fd95ebfe695fd26552|Thor           |M   |
|00088930e925c41fd95ebfe695fd26553|Thor           |G   |
|0009406fd7479715e4bef61dd91f24621|Deadpool       |P   |
|0009406fd7479715e4bef61dd91f24622|Deadpool       |M   |
|0009406fd7479715e4bef61dd91f24623|Deadpool       |G   |
|000b8f95fcb9e0096488278317764d191|Pantera Negra  |P   |
|000b8f95fcb9e0096488278317764d192|Pantera Negra  |M   |
|000b8f95fcb9e0096488278317764d193|Pantera Negra  |G   |
|000d9be29b5207b54e86aa1b1ac548721|Batman         |P   |
|000d9be29b5207b54e86aa1b1ac548722|Batman         |M   |
|000d9be29b5207b54e86aa1b1ac548

In [86]:
path3 = "/content/sheet_3.xlsx - Sheet3.csv"

In [87]:
df3 = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=',').load(path3)

In [93]:
df3.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- Sex: string (nullable = true)



In [88]:
df3.show(20)

+-----+--------------+-----+-----------+------+
|   ID|          City|State|DateOfBirth|   Sex|
+-----+--------------+-----+-----------+------+
|14001|      Curitiba|   PR|  6/28/1985| Homem|
|14002| Florianópolis|   SC|  1/10/1987| Homem|
|14003|Rio de Janeiro|   RJ|  11/5/1979| Homem|
|14004|Rio de Janeiro|   RJ| 11/25/1983|Mulher|
|14005| Florianópolis|   SC|   4/7/1969| Homem|
|14006|Rio de Janeiro|   RJ|  8/17/1951|Mulher|
|14007|     São Paulo|   SP|  1/23/1981| Homem|
|14008|     São Paulo|   SP|  12/7/1971| Homem|
|14009|Rio de Janeiro|   RJ|  12/4/1962|Mulher|
|14010|Rio de Janeiro|   RJ|  9/13/1972| Homem|
|14011|  Porto Alegre|   RS|  11/7/1986| Homem|
|14012| Florianópolis|   SC|  1/25/1983| Homem|
|14013|Rio de Janeiro|   RJ|  11/7/1979| Homem|
|14014|Rio de Janeiro|   RJ|  11/8/1983| Homem|
|14015|Rio piracicaba|   MG|  1/25/1963| Homem|
|14016|      Louveira|   SP|  9/13/1979| Homem|
|14017| Florianópolis|   SC|  6/22/1971| Homem|
|14018|     Sao paulo|   SP|  11/9/1994|

In [89]:
path4 = "/content/sheet_4.xlsx - Sheet4.csv"

In [90]:
df4 = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("inferSchema", True).options(delimiter=',').load(path4)

In [94]:
df4.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- State: string (nullable = true)



In [91]:
df4.show(20)

+---+--------------+-----+
| ID|          Name|State|
+---+--------------+-----+
|  1| Florianópolis|   SC|
|  2|Rio de Janeiro|   RJ|
|  3|  Porto Alegre|   RS|
|  4|      Curitiba|   PR|
|  7|     São Paulo|   SP|
|  9|Belo Horizonte|   MG|
| 22|        Online|   na|
+---+--------------+-----+

